## 단어 빈도를 이용한 벡터화 - Bag of words

In [1]:
doc1 = 'John likes to watch movies. Mary likes movies too.'

In [2]:
BoW = {"too":1, "Mary":1, "movies":2, "John":1, "watch":1, "likes":2, "to":1}

BoW에서 각 key는 단어, 각 value는 각 단어가 주어진 문서에 등장한 횟수입니다. 이때, 문서 내 단어의 순서는 중요하지 않습니다.  
BoW 방식으로 문장을 벡터화한다면 I ate lunch. 와 Lunch ate I. 는 같은 문장이 됩니다. 이렇게 어순에 따라 달라지는 의미를 반영하지 못한다는 점에서 BoW는 한계를 가집니다

### keras Tokenizer 활용

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentence) # 단어장 생성
bow = dict(tokenizer.word_counts) # 각 단어와 각 단어의 빈도를 bow에 저장

print("Bag of Words:", bow) # bow 출력
print('단어장(Vocabulary)의 크기 :', len(tokenizer.word_counts)) # 중복을 제거한 단어들의 개수

Bag of Words: {'john': 1, 'likes': 3, 'to': 2, 'watch': 2, 'movies': 2, 'mary': 2, 'too': 1, 'also': 1, 'football': 1, 'games': 1}
단어장(Vocabulary)의 크기 : 10


단어장의 크기는 10개입니다.

### scikit-learn CountVectorizer 활용
Bag of Words를 만드는 또 다른 방법은 사이킷런의 CountVectorizer를 사용하는 것입니다.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

vector = CountVectorizer()
bow = vector.fit_transform(sentence).toarray()

print('Bag of Words : ', bow) # 코퍼스로부터 각 단어의 빈도수를 기록한다.
print('각 단어의 인덱스 :', vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

Bag of Words :  [[1 1 1 1 3 2 2 2 1 2]]
각 단어의 인덱스 : {'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


In [5]:
print('단어장(Vocabulary)의 크기 :', len(vector.vocabulary_))

단어장(Vocabulary)의 크기 : 10


## 단어 빈도를 이용한 벡터화 - DTM과 코사인 유사도
DTM(Document-Term-Matrix)는 직역함ㄴ 문서-단어 행렬입니다. DTM은 여러 문서의 Bag of Words를 하나의 행렬로 구현한 것입니다. 보통 행을 문서로, 열을 단어로 갖습니다.  
이 때 문서의 수가 많아지면 많아질수록 통합 단어장의 크기도 커지게 되면서 DTM의 문서 벡터와 단어 벡터 대부분의 값이 0이 되는 성질을 가지고 있습니다.

In [8]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

doc1 = np.array([0,1,1,1]) # 문서1 벡터
doc2 = np.array([1,0,1,1]) # 문서2 벡터
doc3 = np.array([2,0,2,2]) # 문서3 벡터

def cos_sim(A,B):
    return dot(A,B)/(norm(A)*norm(B))

In [9]:
print(cos_sim(doc1, doc2)) #문서1과 문서2의 코사인 유사도
print(cos_sim(doc1, doc3)) #문서1과 문서3의 코사인 유사도
print(cos_sim(doc2, doc3)) #문서2과 문서3의 코사인 유사도

0.6666666666666667
0.6666666666666667
1.0000000000000002


문서1과 문서2의 코사인 유사도는 0.67, 문서1과 문서3의 코사인 유사도 또한 0.67입니다. 그런데 문서2와 문서3의 코사인 유사도는 1입니다. 이는 한 문서 내의 모든 단어의 빈도수가 똑같이 증가하는 경우에는 기존의 문서와 코사인 유사도의 값이 1이 되기 때문입니다.

## 단어 빈도를 이용한 벡터화 - DTM의 구현과 한계점
### scikit-learn CountVectorizer 활용
사이킷런의 CountVectorizer를 이용하여 DTM을 만드는 방법은  다수의 문서를 입력값으로 주면 됩니다.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'John likes to watch movies',
    'Mary likes movies too',
    'Mary also likes to watch football games',
]
vector = CountVectorizer()

print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도수를 기록
print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

[[0 0 0 1 1 0 1 1 0 1]
 [0 0 0 0 1 1 1 0 1 0]
 [1 1 1 0 1 1 0 1 0 1]]
{'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


### DTM의 한계점
DTM은 두 가지 한계점을 지닙니다.  
먼저, DTM에서 문서의 수와 단어의 수가 계속 늘어날수록, 행과 열은 대부분의 값이 0을 가진다는 특징이 있습니다. 이는 저장 공간 측면에서 낭비입니다.  
  
두 번째 한계점은 단어의 빈도에만 집중하는 방법 자체의 한계입니다. 예를 들어, 영어 데이터를 가지고 DTM을 만들었을 때, 불용어 'the'는 어떤 문서에서도 자주 등장하는 경향이 있습니다. 그런데 'the'가 자주 등장했다고 해서 두 문서가 유사한 것은 아닙니다.

## 단어 빈도를 이용한 벡터화 - TF-IDF
TF-IDF(Term Frequency-Inverse Document Frequency)는 모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단하며, 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단하는 것입니다.  
하지만 TF-IDF를 사용한다고 하더라도 DTM보다 성능 향상이 뛰어나지 않기 때문에 우선 DTM을 만든 뒤 TF-IDF 가중치를 DTM에 적용합니다.

## 단어 빈도를 이용한 벡터화 - TF-IDF 구현하기
### TF-IDF 구현하기

In [11]:
# 필요한 도구 import
from math import log
import pandas as pd

In [13]:
docs = [
  'John likes to watch movies and Mary likes movies too',
  'James likes to watch TV',
  'Mary also likes to watch football games',  
]

In [14]:
# 통합 단어장 만들기

vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()
print("단어장의 크기:", len(vocab))
print(vocab)

단어장의 크기: 13
['James', 'John', 'Mary', 'TV', 'also', 'and', 'football', 'games', 'likes', 'movies', 'to', 'too', 'watch']


In [15]:
N = len(docs) # 총 문서의 수
N

3

실제 IDF를 구현할 때는 다소 다른 식을 사용합니다. 많은 파이썬 패키지들이 조금씩 조정된 서로 다른 식을 사용합니다.  
여기서는 loglog 항의 분모에 1을 더해주겠습니다. 이는 (Bag of Words를 사용할 때 일반적인 상황은 아니지만) 특정 단어가 전체 문서에서 등장하지 않을 경우에 분모가 0이 되는 상황을 방지하기 위함입니다. 그리고 loglog 항에 1을 더해주겠습니다. 이는 loglog 항의 분자와 분모값이 동일해졌을 때 loglog의 진수가 1이 됨에 따라서 IDF의 값이 0이 되는 것을 방지하기 위함입니다.

In [22]:
def tf(t, d):
    return d.count(t)
 
def idf(t):
    df = 0
    for doc in docs:
        df += t in doc    
    return log(N/(df + 1)) + 1
 
def tfidf(t, d):
    return tf(t,d)* idf(t)

tf 함수를 사용해 DTM을 만듭니다.

In [23]:
result = []
for i in range(N): # 각 문서에 대해서 명령을 수행
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        
        result[-1].append(tf(t,d))
        
tf_ = pd.DataFrame(result, columns = vocab)
tf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0,1,1,0,0,1,0,0,2,2,2,1,1
1,1,0,0,1,0,0,0,0,1,0,1,0,1
2,0,0,1,0,1,0,1,1,1,0,1,0,1


각 단어의 IDF를 구합니다.

In [26]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))
    
idf_ = pd.DataFrame(result, index=vocab, columns=["IDF"])
idf_

,IDF
James,1.405465
John,1.405465
Mary,1.000000
TV,1.405465
also,1.405465
and,1.405465
football,1.405465
games,1.405465
likes,0.712318
movies,1.405465


IDF는 'likes'나 'to' 그리고 'watch'와 같이 모든 문서에 등장한 단어가 가장 낮은 값을 가집니다. 그리고 그다음으로 두 개의 문서에 등장한 'Mary'가 그다음으로 낮은 값을 가지며, 그 외의 1개의 문서에만 등장한 단어들은 가장 높은 값을 가지는 것을 볼 수 있습니다. 이제 TF-IDF 행렬을 출력해봅니다. DTM에 있는 각 단어의 TF에 각 단어의 IDF를 곱해준 값입니다.

In [27]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        
        result[-1].append(tfidf(t,d))
        
tfidf_ = pd.DataFrame(result, columns=vocab)
tfidf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0.000000,1.405465,1.0,0.000000,0.000000,1.405465,0.000000,0.000000,1.424636,2.81093,1.424636,1.405465,0.712318
1,1.405465,0.000000,0.0,1.405465,0.000000,0.000000,0.000000,0.000000,0.712318,0.00000,0.712318,0.000000,0.712318
2,0.000000,0.000000,1.0,0.000000,1.405465,0.000000,1.405465,1.405465,0.712318,0.00000,0.712318,0.000000,0.712318


### scikit-learn TFidVectorizer 활용
사이킷런에서는 TF-IDF를 자동으로 계산하여 출력하는 TfidfVectorizer를 제공합니다.  
사이킷런의 TfidfVectorizer 또한 이번에 배운 기본식에서 조정된 식을 사용합니다. 간단히 언급하면, TfidfVectorizer는 위에서 우리가 파이썬으로 구현한 식에서 더 나아가 loglog 항의 분자에도 1을 더해주며, TF-IDF의 결과에 L2 Norm까지 추가로 수행한다는 점이 다릅니다.

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
  'John likes to watch movies and Mary likes movies too',
  'James likes to watch TV',
  'Mary also likes to watch football games',  
]

tfidfv = TfidfVectorizer().fit(corpus)
vocab = list(tfidfv.vocabulary_.keys()) # 단어장을 리스트로 저장
vocab.sort() # 단어장을 알파벳 순으로 정렬

# TF-IDF 행렬에 단어장을 데이터프레임의 열로 지정하여 데이터프레임 생성
tfidf_ = pd.DataFrame(tfidfv.transform(corpus).toarray(), columns=vocab)
tfidf_

,also,and,football,games,james,john,likes,mary,movies,to,too,tv,watch
0,0.000000,0.321556,0.000000,0.000000,0.000000,0.321556,0.379832,0.244551,0.643111,0.189916,0.321556,0.000000,0.189916
1,0.000000,0.000000,0.000000,0.000000,0.572929,0.000000,0.338381,0.000000,0.000000,0.338381,0.000000,0.572929,0.338381
2,0.464997,0.000000,0.464997,0.464997,0.000000,0.000000,0.274634,0.353642,0.000000,0.274634,0.000000,0.000000,0.274634


## LSA와 LDA - LSA
DTM과 TF-IDF 행렬같이 Bag of Words를 기반으로 한 표현 방법은 근본적으로 단어의 의미를 벡터로 표현하지 못한다는 한계를 가지고 있습니다. LSA(Latent Semantic Analysis)는 이를 해결할 수 있습니다.

### LSA(Latent Semantic Analysis)
잠재 의미 분석이라고 불리는 LSA(Latent Semantic Analysis) 는 전체 코퍼스에서 문서 속 단어들 사이의 관계를 찾아내는 자연어 처리 정보 검색 기술입니다.  
  
LSA를 사용하면 단어와 단어 사이, 문서와 문서 사이, 단어와 문서 사이의 의미적 유사성 점수를 찾아낼 수 있습니다. 그리고 이 방법은 어떤 문서에서 특정 단어들의 빈도가 몇인지를 판단하는 것보다 효과적인 경우가 많습니다.

### 특잇값 분해
특잇값 분해(Singular Value Decomposition, SVD) 란 m × n 크기의 임의의 사각 행렬 A를 위의 그림의 Full SVD와 같이 특이 벡터(singular vector)의 행렬와 특잇값(singular value)의 대각행렬로 분해하는 것을 말합니다.  
특잇값 분해 중에서 조금 특별한 특잇값 분해가 있습니다. 특잇값 가운데 가장 큰(다시 말해, 가장 중요한) t개만 남기고 해당 특잇값에 대응되는 특이 벡터(singular vector) 들로 행렬 A를 근사(approximate) 하도록 하면, 이를 절단된 특잇값 분해(Truncated SVD) 라고 합니다.  
Truncated SVD를 수행하면 행렬 Σ의 대각 원솟값 중에서 상윗값 t개만 남게 되며, U행렬과 V행렬의 t열까지만 남습니다. 이로 인해 세 행렬에서 값(정보)의 손실이 일어나 기존의 행렬 A를 정확히 복구할 수는 없게 됩니다.  
여기서 t는 하이퍼파라미터입니다. 하이퍼파라미터란 사용자가 직접 값을 선택하며 성능에 영향을 주는 매개변수를 말합니다. t를 크게 잡으면 기존의 행렬 A로부터 다양한 의미를 가져갈 수 있지만, 노이즈를 제거하려면 t를 작게 잡아야 합니다.

### LSA와 Truncated SVD
LSA는 DTM이나 TF-IDF 행렬 등에 Truncated SVD를 수행합니다. 이렇게 분해하여 얻은 행렬 3개(S,V,S)는 각각 '문서들과 관련된 의미들을 표현한 행렬', '단어들과 관련된 의미를 표현한 행렬' , '각 의미의 중요도를 표현한 행렬' 이라고 해석할 수 있습니다. 하이퍼파라미터는 k입니다.  

In [41]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [42]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /aiffel/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /aiffel/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /aiffel/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

데이터 다운로드 및 확인

In [44]:
import os

csv_filename = os.getenv('HOME')+'/aiffel/topic_modelling/data/abcnews-date-text.csv'

urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv",
filename=csv_filename)

('/aiffel/aiffel/topic_modelling/data/abcnews-date-text.csv',
 <http.client.HTTPMessage at 0x7f7215f7eb90>)

데이터프레임에 저장 밑 전체 샘플의 수 출력

In [45]:
data = pd.read_csv(csv_filename, error_bad_lines=False)
data.shape

(1082168, 2)

In [46]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


headline_text만 저장

In [47]:
text = data[['headline_text']].copy()
text.head()

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


In [49]:
# 중복 확인
text.nunique()

headline_text    1054983
dtype: int64

약 108만개 중 중복을 제외하면 105만개의 샘플이 존재.

In [50]:
# 중복 제거
text.drop_duplicates(inplace=True) # 중복 샘플 제거
text.reset_index(drop=True, inplace=True)
text.shape

(1054983, 1)

### 데이터 정제 및 정규화
NLTK의 토크나이저를 이용해 전체 텍스트 데이터에 대해서 단어 토큰화를 수행  
NLTK가 제공하는 불용어 리스트를 사용하여 불용어를 제거

In [52]:
# NLTK 토크나이저를 이용해서 토큰화
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

# 불용어 제거
stop_words = stopwords.words('english')
text['headline_text']=text['headline_text'].apply(lambda x:[word for word in x if word not in (stop_words)])

text.head()

,headline_text
0,"[aba, decides, community, broadcasting, licence]"
1,"[act, fire, witnesses, must, aware, defamation]"
2,"[g, calls, infrastructure, protection, summit]"
3,"[air, nz, staff, aust, strike, pay, rise]"
4,"[air, nz, strike, affect, australian, travellers]"


동일한 단어지만 다른 표현을 가지는 단어들은 하나의 단어로 통합(lemmatization)하는 단어 정규화, 그리고 길이가 1~2인 단어를 제거하는 전처리를 진행합니다.

In [54]:
# 단어 정규화. 3인칭 단수 표현 -> 1인칭 변환, 과거형 동사 -> 현재형 동사 등을 수행합니다.
text['headline_text'] = text['headline_text'].apply(lambda x:[WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

# 길이가 1~2인 단어는 제거.
text = text['headline_text'].apply(lambda x:[word for word in x if len(word) > 2])
print(text[:5])

0     [aba, decide, community, broadcast, licence]
1    [act, fire, witness, must, aware, defamation]
2       [call, infrastructure, protection, summit]
3            [air, staff, aust, strike, pay, rise]
4    [air, strike, affect, australian, travellers]
Name: headline_text, dtype: object


### 역토큰화 및 DTM 생성
DTM을 생성하는 CountVectorizer 또는 TF-IDF 행렬을 생성하는 TfidfVectorizer의 입력으로 사용하기 위해서 토큰화 과정을 역으로 되돌리는 역토큰화(detokenization)를 수행합니다.

In [57]:
# 역토큰화(토큰화 작업을 역으로 수행)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(text[i])
    detokenized_doc.append(t)
    
train_data = detokenized_doc

In [58]:
train_data[:5]

['aba decide community broadcast licence',
 'act fire witness must aware defamation',
 'call infrastructure protection summit',
 'air staff aust strike pay rise',
 'air strike affect australian travellers']

CountVectorizer를 사용하여 DTM을 생성합니다. 단어의 수는 5,000개로 제한합니다.

In [60]:
c_vectorizer = CountVectorizer(stop_words='english',max_features=5000)
document_term_matrix = c_vectorizer.fit_transform(train_data)

In [61]:
print('행렬의 크기:', document_term_matrix.shape)

행렬의 크기: (1054983, 5000)


DTM의 크기(shape)는 (문서의 수 × 단어 집합의 크기)입니다.

### scikit-learn TruncatedSVD 활용
 Truncated SVD를 통해 LSA를 수행합니다. 토픽의 수를 10으로 정하겠습니다.

In [63]:
from sklearn.decomposition import TruncatedSVD

n_topics=10
lsa_model = TruncatedSVD(n_components = n_topics)
lsa_model.fit_transform(document_term_matrix)

array([[ 1.20107303e-02, -3.76110139e-03,  1.82705773e-02, ...,
         8.20031196e-03,  2.85748132e-03,  1.60805845e-02],
       [ 2.90376208e-02, -1.08904385e-02,  1.82314359e-02, ...,
         4.88610282e-03, -1.22211881e-02, -6.33398311e-04],
       [ 4.99441886e-03, -2.16827789e-03,  9.79048688e-03, ...,
        -3.83721454e-04,  1.50604878e-03,  3.91046158e-03],
       ...,
       [ 2.95882541e-02,  3.92588568e-03,  2.53212716e-02, ...,
         4.69102505e-02,  2.50282248e-02,  1.59827606e-02],
       [ 6.14492092e-02, -6.99244274e-03,  1.33574566e-01, ...,
         8.80336328e-01,  9.66047741e-01, -3.09321232e-01],
       [ 7.17403175e-02,  3.10839675e-02, -4.61848844e-04, ...,
        -3.19604069e-03,  9.62132170e-03, -1.25651068e-02]])

TruncatedSVD를 통해 얻은 행렬 V의 크기를 확인해봅니다.

행렬 V가 k*(단어의 수) 크기를 갖는 것을 확인할 수 있습니다.  
각 행을 전체 코퍼스의 k개의 주제(topic)로 판단하고 각 주제에서 n개씩 단어를 출력해 봅니다.

In [68]:
terms = c_vectorizer.get_feature_names() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lsa_model.components_, terms)

Topic 1: [('police', 0.7464), ('man', 0.45353), ('charge', 0.21092), ('new', 0.14091), ('court', 0.11158)]
Topic 2: [('man', 0.69409), ('charge', 0.30017), ('court', 0.16835), ('face', 0.1158), ('murder', 0.10612)]
Topic 3: [('new', 0.83667), ('plan', 0.23667), ('say', 0.18268), ('govt', 0.1106), ('council', 0.10988)]
Topic 4: [('say', 0.73807), ('plan', 0.35935), ('govt', 0.16801), ('council', 0.131), ('fund', 0.07375)]
Topic 5: [('plan', 0.73404), ('council', 0.16774), ('govt', 0.13334), ('urge', 0.08792), ('water', 0.06882)]
Topic 6: [('govt', 0.55514), ('court', 0.23889), ('fund', 0.23684), ('urge', 0.23233), ('face', 0.14824)]
Topic 7: [('charge', 0.53142), ('court', 0.4531), ('face', 0.34276), ('murder', 0.13405), ('plan', 0.11466)]
Topic 8: [('win', 0.49134), ('court', 0.42694), ('australia', 0.15189), ('crash', 0.1188), ('cup', 0.09033)]
Topic 9: [('win', 0.65644), ('charge', 0.41054), ('australia', 0.10546), ('cup', 0.08703), ('world', 0.07912)]
Topic 10: [('council', 0.81029)

## LSA와 LDA - LDA
문서의 집합에서 토픽을 찾아내는 프로세스를 토픽 모델링(Topic Modelling) 이라고 합니다. 이는 고객의 소리와 같이 많은 문서에서 주요 주제를 알아내는 일이 중요할 때 사용합니다.  
잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)은 토픽 모델링의 또 다른 대표적인 알고리즘입니다.  
LDA는 문서들이 토픽들의 혼합으로 구성되어 있으며 토픽들은 확률 분포에 기반하여 단어들을 생성한다고 가정합니다. 이 가정에 따라 단어들의 분포로부터 문서가 생성되는 과정을 역추적해 문서의 토픽을 찾아냅니다.

### LDA(Latent Dirichlet Allocation)의 두 가지 결과
LDA는 각 토픽의 단어 분포와 각 문서의 토픽 분포를 추정해냅니다.  

### LDA의 가정
LDA는 전체 코퍼스, 즉 다수의 문서들로부터 토픽을 뽑아내기 위해서 하나의 가정을 염두에 두고 있습니다. 모든 문서 하나 하나가 작성될 때 그 문서의 작성자는 아래와 같은 생각을 했다는 가정입니다.  
'나는 이 **문서를** 작성하기 위해서 이런 **주제**들을 넣을거고,  
이런 **주제**들을 위해서는 이런 **단어**들을 넣을 거야.'

### TF-IDF 행렬 생성
LDA는 DTM 또는 TF-IDF를 입력으로 받을 수 있습니다. 여기서는 TF-IDF를 사용하겠습니다.  
TfidfVectorizer를 사용하여 TF-IDF 행렬을 생성합니다. 단어의 수는 5,000개로 제한합니다.

In [69]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tf_idf_matrix = tfidf_vectorizer.fit_transform(train_data)

print("행렬의 크기:", tf_idf_matrix.shape)

행렬의 크기: (1054983, 5000)


### scikit-learn LDA Model 활용

In [71]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', random_state=777, max_iter=1)
lda_model.fit_transform(tf_idf_matrix)

array([[0.0335099 , 0.69841093, 0.0335099 , ..., 0.0335099 , 0.0335099 ,
        0.0335099 ],
       [0.03365631, 0.03365631, 0.03365631, ..., 0.03365631, 0.03365631,
        0.03365631],
       [0.0366096 , 0.0366096 , 0.0366096 , ..., 0.67051361, 0.0366096 ,
        0.0366096 ],
       ...,
       [0.02914502, 0.02914502, 0.14077174, ..., 0.02914502, 0.02914502,
        0.26688721],
       [0.02637829, 0.12325014, 0.02638944, ..., 0.21422895, 0.02637829,
        0.0996168 ],
       [0.03376121, 0.03376055, 0.03376055, ..., 0.03376055, 0.50437083,
        0.03376055]])

In [72]:
print(lda_model.components_.shape)

(10, 5000)


In [75]:
# LDA의 결과 토픽과 각 단어의 비중을 출력합니다.
terms = tfidf_vectorizer.get_feature_names() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n-1 :-1]])
        
get_topics(lda_model.components_, terms)

Topic 1: [('new', 6839.37079), ('government', 6344.47105), ('election', 5419.59529), ('adelaide', 4864.1739), ('home', 4048.41226)]
Topic 2: [('say', 7929.44145), ('change', 4193.89405), ('year', 3924.88995), ('live', 3625.10473), ('market', 3541.15301)]
Topic 3: [('australian', 7667.75985), ('south', 4846.36918), ('perth', 4552.99622), ('2016', 3955.96018), ('open', 3771.68108)]
Topic 4: [('trump', 8187.32772), ('school', 3966.83143), ('jail', 3245.22756), ('women', 3029.28982), ('life', 2998.44275)]
Topic 5: [('police', 5589.78951), ('melbourne', 5299.84238), ('warn', 3577.31091), ('rural', 3521.5736), ('hospital', 3106.7779)]
Topic 6: [('world', 4536.54893), ('sydney', 4406.58731), ('country', 4167.71984), ('years', 3581.99631), ('man', 3520.89397)]
Topic 7: [('charge', 5946.75892), ('day', 5062.31785), ('house', 4481.76928), ('murder', 4065.57534), ('crash', 3793.50261)]
Topic 8: [('australia', 7253.84683), ('attack', 4787.62503), ('north', 3706.11788), ('state', 3658.17043), ('wes

## 텍스트 분포를 이용한 비지도 학습 토크나이저 - 형태소 분석기와 단어 미등록 문제
지금까지 띄어쓰기 단위의 단어 분포를 이용해서 단어나 문서를 벡터화하는 방법에 대해 알아봤습니다. 이번에는 텍스트의 분포를 이용해서 토큰화를 수행하는 비지도 학습 토크나이저에 대해서 알아봅시다.

### 형태소 분석기의 필요성
한국어는 교착어입니다. 교착어란 하나의 낱말(엄밀히는 하나의 어절)이 하나의 어근(root)(혹은 어간(stem))과 각각 단일한 기능을 가지는 하나 이상의 접사(affix)의 결합으로 이루어져 있는 언어를 말합니다. 조사라는 품사는 교착어에만 존재합니다.  
이런 특성으로 인해 한국어는 영어와 달리 조사나 접사가 존재하며, 영어처럼 띄어쓰기 단위 토큰화가 제대로 동작하지 않습니다.

In [76]:
en_text = "The dog ran back to the corner near the spare bedrooms"
print(en_text.split())

['The', 'dog', 'ran', 'back', 'to', 'the', 'corner', 'near', 'the', 'spare', 'bedrooms']


한국어의 경우에는 띄어쓰기 단위로 토큰화를 하면 문제가 생깁니다.

In [77]:
kor_text = "사과의 놀라운 효능이라는 글을 봤어. 그래서 오늘 사과를 먹으려고 했는데 사과가 썩어서 슈퍼에 가서 사과랑 오렌지 사 왔어"
print(kor_text.split())

['사과의', '놀라운', '효능이라는', '글을', '봤어.', '그래서', '오늘', '사과를', '먹으려고', '했는데', '사과가', '썩어서', '슈퍼에', '가서', '사과랑', '오렌지', '사', '왔어']


위 문장에서 '사과'는 총 4번 등장했는데 모두 접사가 붙어 각기 다른 단어로 인식될 수 있습니다. 이를 해결하기 위해 한국어는 주로 형태소 분석기를 사용합니다.

In [78]:
from konlpy.tag import Okt

tokenizer = Okt()
print(tokenizer.morphs(kor_text))

['사과', '의', '놀라운', '효능', '이라는', '글', '을', '봤어', '.', '그래서', '오늘', '사과', '를', '먹으려고', '했는데', '사과', '가', '썩어서', '슈퍼', '에', '가서', '사과', '랑', '오렌지', '사', '왔어']


### 단어 미등록 문제
형태로 분석기는 등록된 단어를 기준으로 형태소를 분류하기 때문에 새롭게 만들어진 단어를 인식하기 어렵다는 특징이 있습니다.

In [79]:
print(tokenizer.morphs('모두의연구소에서 자연어 처리를 공부하는 건 정말 즐거워'))

['모두', '의', '연구소', '에서', '자연어', '처리', '를', '공부', '하는', '건', '정말', '즐거워']


'모두의연구소'는 하나의 단어이지만 위의 형태소 분석 결과에서는 전부 분리된 결과를 보여줍니다.  
그렇다면 텍스트 데이터에서 특정 문자 시퀀스가 함께 자주 등장하는 빈도가 높고, 앞뒤로 조사 또는 완전히 다른 단어가 등장하는 것을 고려해서 해당 문자 시퀀스를 형태소라고 판단하는 형태소 분석기라면 어떨까요?  
예를 들어, '모두의연구소'라는 문자열이 자주 연결되어 등장한다면 형태소라고 판단하고, '모두의연구소'라는 단어 앞, 뒤에 '최고', 'AI', '실력'과 같은 독립된 다른 단어들이 계속해서 등장한다면 '모두의연구소'를 형태소로 파악하는 식이지요. 이런 아이디어를 가진 형태소 분석기가 soynlp입니다.

## 텍스트 분포를 이용한 비지도 학습 토크나이저 - soynlp
soynlp는 품사 태깅, 형태소 분석 등을 지원하는 한국어 형태소 분석기입니다.
비지도 학습으로 형태소 분석을 한다는 특징을 갖고 있으며, 데이터에 자주 등장하는 단어들을 형태소로 분석합니다.  
soynlp 형태소 분석기는 내부적으로 단어 점수표로 동작합니다. 이 점수는 응집 확률(cohesion probability) 과 브랜칭 엔트로피(branching entropy) 를 활용합니다.

In [80]:
# soynlp 깃허브에서 제공하는 예제 말뭉치를 다운로드 합니다.
import urllib.request

txt_filename = os.getenv('HOME')+'/aiffel/topic_modelling/data/2016-10-20.txt'

urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt",\
                          filename=txt_filename)

('/aiffel/aiffel/topic_modelling/data/2016-10-20.txt',
 <http.client.HTTPMessage at 0x7f72091f66d0>)

In [82]:
from soynlp import DoublespaceLineCorpus

# 말뭉치에 대해서 다수의 문서로 분리
corpus = DoublespaceLineCorpus(txt_filename)
len(corpus)

30091

총 30,091개의 문서가 존재합니다. 공백이 아닌 문서에 한해 상위 3개의 문서만 출력합니다.

In [83]:
i = 0
for document in corpus:
    if len(document) > 0:
        print(document)
        i = i+1
        if i ==3:
            break

19  1990  52 1 22
오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에

soynlp는 비지도학습 형태소 분석기이므로 기존의 형태소 분석기와는 달리 학습 과정을 거쳐야 합니다. 전체 코퍼스로부터 응집 확률과 브랜칭 엔트로피 단어 점수표를 만드는 과정입니다. WordExtractor.extract()를 통해서 전체 코퍼스에 대해 단어 점수표를 계산할 수 있습니다.

In [85]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 2.797 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


### soynlp의 응집 확률(cohesion probability()
응집 확률은 내부 문자열(substring)이 얼마나 응집하여 자주 등장하는지를 판단하는 척도입니다. 응집 확률은 문자열을 문자 단위로 분리하여 내부 문자열을 만드는 과정에서, 왼쪽부터 순서대로 문자를 추가하면서 각 문자열이 주어졌을 때 그다음 문자가 나올 확률을 계산하여 누적 곱을 한 값입니다. 이 값이 높을수록 전체 코퍼스에서 이 문자열 시퀀스는 하나의 단어로 등장할 가능성이 높습니다.

In [89]:
word_score_table["반포한"].cohesion_forward

0.08838002913645132

In [90]:
word_score_table["반포한강공"].cohesion_forward

0.2972877884078849

In [91]:
word_score_table["반포한강공원"].cohesion_forward

0.37891487632839754

In [92]:
word_score_table["반포한강공원에"].cohesion_forward

0.33492963377557666

응집 확률이 최대화가 되는 내부 문자열 구간은 '반포한강공원'입니다. 다시 말해 '반포한강공원'이 하나의 단어일 확률이 높습니다.

### soynlp의 브랜칭 엔트로피(branching entropy)
브랜칭 엔트로피(Branching Entropy)는 확률 분포의 엔트로피 값을 사용합니다. 이는 주어진 문자열에서 다음 문자가 등장할 수 있는 가능성을 판단하는 척도입니다.  
브랜칭 엔트로피를 주어진 문자 시퀀스에서 다음 문자 예측을 위해 헷갈리는 정도라고 비유해 봅시다. 브랜칭 엔트로피의 값은 하나의 완성된 단어에 가까워질수록 문맥으로 인해 정확히 예측할 수 있게 되므로 점차 줄어듭니다.

In [93]:
word_score_table["디스"].right_branching_entropy

1.6371694761537934

In [94]:
word_score_table["디스플"].right_branching_entropy

-0.0

'디스' 다음에는 다양한 문자가 올 수 있으니까 1.63이라는 값을 가지는 반면, '디스플'이라는 문자열 다음에는 다음 문자로 '레'가 오는 것이 너무나 명백하기 때문에 0이란 값을 가집니다.

In [95]:
word_score_table["디스플레"].right_branching_entropy

-0.0

'디스플레' 다음도 마찬가지입니다.

In [96]:
word_score_table["디스플레이"].right_branching_entropy

3.1400392861792916

갑자기 값이 급증합니다. 그 이유는 문자 시퀀스 '디스플레이'라는 문자 시퀀스 다음에는 조사나 다른 단어가 나올 수 있는 다양한 경우가 있을 수 있기 때문입니다. 즉 하나의 단어가 끝나면 그 경계 부분부터 다시 브랜칭 엔트로피값이 증가함을 의미합니다.  
soynlp 패키지는 이렇게 계산한 점수를 사용해 두 가지 문자열 토큰화 방법을 제공합니다.

### soynlp의 LTokenizer
띄어쓰기 단위로 잘 나뉜 문장은 L 토크나이저(LTokenizer)를 사용하면 좋습니다.  
한국어는 띄어쓰기 단위로 나눈 어절 토큰이 주로 L 토큰 + R 토큰의 형식을 가질 때가 많습니다. 예를 들어서 '공원에'는 '공원 + 에'로 나눌 수 있겠지요. 또는 '공부하는'은 '공부 + 하는'으로 나눌 수도 있을 것입니다. L 토크나이저는 L 토큰 + R 토큰으로 나누되, 점수가 가장 높은 L 토큰을 찾아내는 분리 기준을 가지고 있습니다.

In [98]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

### 최대 점수 토크나이저
최대 점수 토크나이저(MaxScoreTokenizer)는 띄어쓰기가 되어 있지 않은 문장에서 점수가 높은 글자 시퀀스를 순차적으로 찾아내는 토크나이저입니다. 띄어쓰기가 되어 있지 않은 문장을 넣어서 토큰화된 결과를 점수를 통해 알아보겠습니다.

In [99]:
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']